In [1]:
import tensorflow as tf
import numpy as np
from __future__ import print_function

A similar algorithm is commonly referred to as the Viterbi algorithm, but also known as a special case of the max-product or min-sum algorithm, which solves the related problem of maximization, or most probable explanation. Instead of attempting to solve the marginal, the goal here is to find the values \mathbf{x} that maximises the global function (i.e. most probable values in a probabilistic setting), and it can be defined using the arg max:

\operatorname*{arg\,max}_{\mathbf{x}} g(\mathbf{x}).
An algorithm that solves this problem is nearly identical to belief propagation, with the sums replaced by maxima in the definitions.[8]

It is worth noting that inference problems like marginalization and maximization are NP-hard to solve exactly and approximately (at least for relative error) in a graphical model. More precisely, the marginalization problem defined above is #P-complete and maximization is NP-complete.

The memory usage of belief propagation can be reduced through the use of the Island algorithm (at a small cost in time complexity).

In [18]:
def dptable(state_prob):
    print(" ".join(("%10d" % i) for i in range(state_prob.shape[0])))
    for i, prob in enumerate(state_prob.T):
        print("%.7s: " % states[i] +" ".join("%.7s" % ("%f" % p) for p in prob))

In [34]:
class HiddenMarkovModel(object):

    """
    Hidden Markov Model Class

    Parameters:
    -----------
    
    - S: Number of states.
    - T: Transition matrix of size S by S
         stores probability from state i to state j.
    - E: Emission matrix of size S by N (number of observations)
         stores the probability of observing  O_j  from state  S_i. 
    - T0: Initial state probabilities of size S.
    """

    def __init__(self, T, E, T0):
        # Number of states
        self.S = T.shape[0]
        
        # Emission probability
        self.E = tf.constant(E, name='emission_matrix')

        # Transition matrix
        self.T = tf.constant(T, name='transition_matrix')

        # Initial state vector
        self.T0 = tf.constant(T0, name='inital_state_vector')

    def initialize_path_variables(self, shape):
        
        pathStates = tf.Variable(tf.zeros(shape, dtype=tf.int64), name='States_matrix')
        pathScores = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='Score_matrix')
        states_seq = tf.Variable(tf.zeros([shape[0]], dtype=tf.int64), name='States_sequence')
        return pathStates, pathScores, states_seq
    
    def belief_propagation(self, scores):
        
        scores_reshape = tf.reshape(scores, (-1,1))
        return tf.add(scores_reshape, tf.log(self.T))
    
    def viterbi_inference(self, obs_seq):
        
        # length of observed sequence
        self.N = len(obs_seq)
        
        # shape path Variables
        shape = [self.N, self.S]
        
        # observed sequence
        x = tf.constant(obs_seq, name='observation_sequence')
        
        # Initialize variables
        pathStates, pathScores, states_seq = self.initialize_path_variables(shape)       
        
        # log probability of emission sequence
        obs_prob_seq = tf.log(tf.gather(self.E, x))
        obs_prob_list = tf.split(0, self.N, obs_prob_seq)

        # step 1 state log-prior update
        pathScores = tf.scatter_update(pathScores, 0, tf.log(self.T0) + tf.squeeze(obs_prob_list[0]))
            
        
        for step, obs_prob in enumerate(obs_prob_list[1:]):
            
            # propagate state belief
            belief = self.belief_propagation(pathScores[step, :])

            # the inferred state by maximizing global function
            # and update state and score matrices 
            pathStates = tf.scatter_update(pathStates, step + 1, tf.argmax(belief, 0))
            pathScores = tf.scatter_update(pathScores, step + 1, tf.reduce_max(belief, 0) + tf.squeeze(obs_prob))

        # infer most likely last state
        states_seq = tf.scatter_update(states_seq, self.N-1, tf.argmax(pathScores[self.N-1, :], 0))
        
        for step in range(self.N - 1, 0, -1):
            # for every timestep retrieve inferred state
            state = states_seq[step]
            idx = tf.reshape(tf.pack([step, state]), [1, -1])
            state_prob = tf.gather_nd(pathStates, idx)
            states_seq = tf.scatter_update(states_seq, step - 1,  state_prob[0])

        return states_seq, tf.exp(pathScores) # turn scores back to probabilities
    
    def run_viterbi(self, obs_seq):
        with tf.Session() as sess:
            
            state_graph, state_prob_graph = self.viterbi_inference(obs_seq)
            sess.run(tf.initialize_all_variables())
            states_seq, state_prob = sess.run([state_graph, state_prob_graph])

        return states_seq, state_prob 
    
    
    def initialize_variables(self, shape, shape_ext):
        self.forward = tf.Variable(tf.zeros(shape_ext, dtype=tf.float64), name='forward')
        self.backward = tf.Variable(tf.zeros(shape_ext, dtype=tf.float64), name='backward')
        self.posterior = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='posteriror')


    def _forward(self, obs_prob_seq):
        # initialize with state probabilities T0
        self.forward = tf.scatter_update(self.forward, 0, self.T0)

        for step in range(self.N):
            prev_prob = tf.reshape(self.forward[step, :], [1, -1])
            prior_prob = tf.matmul(prev_prob, self.T)
            # forward belief propagation
            forward_score = tf.mul(prior_prob, tf.cast(obs_prob_seq[step, :], tf.float64))
            # Normalize score into a probability
            forward_prob = tf.reshape(forward_score / tf.reduce_sum(forward_score), [-1])
            # Update forward matrix
            self.forward = tf.scatter_update(self.forward, step + 1, forward_prob)

        # remove initial probabilities
        self.forward = tf.slice(self.forward, [1,0], [5,2]) 
        

    def _backward(self, obs_prob_seq):
        # backward pass
        self.backward = tf.scatter_update(self.backward, self.N, tf.ones([self.S], dtype=tf.float64)) 

        for step in range(self.N, 0, -1):
            next_prob = tf.reshape(self.backward[step, :], [-1, 1])
            obs_prob = tf.diag(obs_prob_seq[step - 1, :])
            prior_prob = tf.matmul(self.T, obs_prob)
            backward_score = tf.matmul(prior_prob, next_prob)
            backward_prob = tf.reshape(backward_score / tf.reduce_sum(backward_score), [-1])

            # Update backward matrix
            self.backward = tf.scatter_update(self.backward, step - 1, backward_prob)
        
        # remove final probabilities
        self.backward = tf.slice(self.backward, [0,0], [5,2])

        
    def forward_backward(self, obs_seq):
        """
        runs forward backward algorithm on state probabilities y

        Arguments
        ---------
        y : np.array : shape (T, K) where T is number of timesteps and
            K is the number of states

        Returns
        -------
        (posterior, forward, backward)
        posterior : list of length T of tensorflow graph nodes representing
            the posterior probability of each state at each time step
        forward : list of length T of tensorflow graph nodes representing
            the forward probability of each state at each time step
        backward : list of length T of tensorflow graph nodes representing
            the backward probability of each state at each time step
        """

        # length of observed sequence
        self.N = len(obs_seq)

        # shape path Variables
        shape = [self.N, self.S]
        shape_ext = [self.N+1, self.S]
        
        # observed sequence
        x = tf.constant(obs_seq, dtype=tf.int32, name='observation_sequence')
        
        self.initialize_variables(shape, shape_ext)
        
        obs_prob_seq = tf.gather(self.E, x)
        
        self._forward(obs_prob_seq)
        
        self._backward(obs_prob_seq)

        # posterior score
        self.posterior = tf.mul(self.forward, self.backward)
        
        # marginal per timestep
        marginal = tf.reduce_sum(self.posterior, 1)
        
        # Normalize porsterior into probabilities
        self.posterior = self.posterior / tf.reshape(marginal, [-1, 1])

        return self.forward, self.backward, self.posterior
    
    def run_forward_backward(self, obs_seq):
        with tf.Session() as sess:
            
            forward, backward, posterior = self.forward_backward(obs_seq)
            sess.run(tf.initialize_all_variables())
            return sess.run([forward, backward, posterior])

In [43]:
p0 = np.array([0.6, 0.4])

emi = np.array([[0.5, 0.1],
                [0.4, 0.3],
                [0.1, 0.6]])

trans = np.array([[0.7, 0.3],
                  [0.4, 0.6]])

states = {0:'Healthy', 1:'Fever'}
obs = {0:'normal', 1:'cold', 2:'dizzy'}

obs_seq = np.array([0, 1, 2])

In [44]:
model =  HiddenMarkovModel(trans, emi, p0)
states_seq, state_prob = model.run_viterbi(obs_seq)
dptable(state_prob)
print()
print("Most likely States: ", [states[s] for s in states_seq])

         0          1          2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512

Most likely States:  ['Healthy', 'Healthy', 'Fever']


In [45]:
p0 = np.array([0.5, 0.5])

emi = np.array([[0.9, 0.2],
                [0.1, 0.8]])

trans = np.array([[0.7, 0.3],
                  [0.3, 0.7]])

states = {0:'rain', 1:'no_rain'}
obs = {0:'umbrella', 1:'no_umbrella'}

obs_seq = np.array([0, 0, 1, 0, 0])

In [49]:
model =  HiddenMarkovModel(trans, emi, p0)

results = model.run_forward_backward(obs_seq)
result_list = ["Forward", "Backward", "Posterior"]

for state_prob, path in zip(results, result_list) :
    inferred_states = np.argmax(state_prob, axis=1)
    print()
    print(path)
    print("Most likely States: ",[states[s] for s in inferred_states])
    print()
    dptable(state_prob)
    print()


Forward
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.81818 0.88335 0.19066 0.73079 0.86733
no_rain: 0.18181 0.11664 0.80933 0.26920 0.13266


Backward
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.64693 0.59231 0.37626 0.65334 0.62727
no_rain: 0.35306 0.40768 0.62373 0.34665 0.37272


Posterior
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.89184 0.91668 0.12443 0.83650 0.91668
no_rain: 0.10816 0.08331 0.87556 0.16349 0.08331

